## MCMC and POPGEN

In [ ]:
ENV["PYTHON"] = "" 
using Pkg
Pkg.build("PyCall")  # Rebuild PyCall to use the internal Python
using Revise, Genie, DelimitedFiles, DCAUtils, JLD2, PyPlot, Statistics, LinearAlgebra
import KitMSA: fasta2matrix, matrix2fasta


h_tmp, J_tmp = read_par_BM_lettersave("../model_natural_PSE.dat");
h = set_max_field_to_0save(h_tmp);
J_tmp2 = symmetrize_Jsave(J_tmp); 
J = permutedims(J_tmp2, [1,3,2,4]);


h_tmp, J_tmp = read_par_BM_0gapsave("../data_directed_evolution/Parameters_conv_AAC6.dat");
h_aac = set_max_field_to_0save(h_tmp);
J_tmp2 = symmetrize_Jsave(J_tmp); 
J_aac = permutedims(J_tmp2, [1,3,2,4]);


wt_tem = join(readdlm("../Gen.jl/data/alignments/refs/TEM1_pfam_DNA.fasta", skipstart = 1));
L_big = round(Int,length(wt_tem)/3)
tem_DNA_seq = [wt_tem[((i-1)*3 +1):(i*3)] for i in 3:L_big-1];
tem = [cod2amino[x] for x in tem_DNA_seq];



wt_pse = join(readdlm("../Gen.jl/data/alignments/refs/PSE1_pfam_DNA.fasta", skipstart = 1));
L_big = round(Int,length(wt_pse)/3)
pse_DNA_seq = [wt_pse[((i-1)*3 +1):(i*3)] for i in 3:L_big-1];


wt_aac = join(readdlm("../Gen.jl/data/alignments/refs/AAC6_DNA_pfam.fasta", skipstart = 1));
L_big_aac = round(Int,length(wt_aac)/3)
aac_DNA_seq = [wt_aac[((i-1)*3 +1):(i*3)] for i in 1:L_big_aac];
aac = [cod2amino[x] for x in aac_DNA_seq];


@load "../data_directed_evolution/data_dir_evol_amino_tem.jld2"


@load "../data_directed_evolution/data_dir_evol_amino_pse.jld2"


@load "../data_directed_evolution/data_dir_evol_amino_aac.jld2"
    


In [ ]:
############### STIFFLER #################


start_msa = hcat([pse_DNA_seq for i in 1:10^6]...); 
    @time mcmc20 = run_evolution(start_msa, 
    h, 
    J,
    p = 0., 
    temp = 1.30, 
    N_steps = 125,  
    verbose = false);


start_msa = hcat([pse_DNA_seq for i in 1:10^6]...); 
    @time mcmc10 = run_evolution(start_msa, 
    h, 
    J,
    p = 0., 
    temp = 1.30, 
    N_steps = 62,  
    verbose = false);

N_start = 10^6; @time pop_gen20 = Genie.run_dir_evol_nucleo(pse_DNA_seq, N_start, h, J;
                   rounds = 20,
                   seq_reads = N_start,
                   temp = 10.,  
                   mu = 0.025, mu_bind = -35.);

N_start = 10^6; @time pop_gen10 = Genie.run_dir_evol_nucleo(pse_DNA_seq, N_start, h, J;
                   rounds = 10,
                   seq_reads = N_start,
                   temp = 10.,  
                   mu = 0.025, mu_bind = -35.);




n_seq_stiffler = 4000;

@time stiffler_exp = [Genie.analyze_target_msa(pse_dir_msa_10, pse, h, J, 10; n_seq = n_seq_stiffler),
    analyze_target_msa(pse_dir_msa_20, pse, h, J, 20; n_seq = n_seq_stiffler)];

stiffler_mcmc = [Genie.analyze_msa(mcmc10.msa, pse_dir_msa_10, pse, h, J, 10; n_seq = n_seq_stiffler),
    analyze_msa(mcmc20.msa, pse_dir_msa_20, pse, h, J, 20; n_seq = n_seq_stiffler)];

stiffler_popgen = [Genie.analyze_msa(pop_gen10.final_msa, pse_dir_msa_10, pse, h, J, 10; n_seq = n_seq_stiffler),
    analyze_msa(pop_gen20.final_msa, pse_dir_msa_20, pse, h, J, 20; n_seq = n_seq_stiffler)];


########## AAC ################



start_msa = hcat([aac_DNA_seq for i in 1:10^6]...); 
    @time mcmc8 = run_evolution(start_msa, 
    h_aac, 
    J_aac,
    p = 0., 
    temp = 1.7, 
    N_steps = 45,  
    verbose = false);


N_start = 3*10^5; @time pop_gen8 = Genie.run_dir_evol_nucleo(aac_DNA_seq, N_start, h_aac, J_aac;
                   rounds = 8,
                   seq_reads = N_start,
                   temp = 6.,  
                   mu = 0.03, mu_bind = 18.);

n_seq_aac = 40000; 

@time aac_exp = Genie.analyze_target_msa(aac_dir_msa_8, aac, h_aac, J_aac, 8; n_seq = n_seq_aac);
    
aac_mcmc = Genie.analyze_msa(mcmc8.msa, aac_dir_msa_8, aac, h_aac, J_aac, 8; n_seq = n_seq_aac);

aac_popgen = Genie.analyze_msa(pop_gen8.final_msa, aac_dir_msa_8, aac, h_aac, J_aac, 8; n_seq = n_seq_aac);



########### FANTINI  ##############à


start_msa = hcat([tem_DNA_seq for i in 1:3*10^5]...); 
    @time mcmc12 = run_evolution(start_msa, 
    h, 
    J,
    p = 0., 
    temp = 1.14, 
    N_steps = 95,  
    verbose = false);

start_msa = hcat([tem_DNA_seq for i in 1:3*10^5]...); 
    @time mcmc6 = run_evolution(start_msa, 
    h, 
    J,
    p = 0., 
    temp = 1.14, 
    N_steps = 48,  
    verbose = false);



N_start = 3*10^5; @time pop_gen12 = Genie.run_dir_evol_nucleo(tem_DNA_seq, N_start, h, J;
                   rounds = 12,
                   seq_reads = N_start,
                   temp = 8.,  
                   mu = 0.03, mu_bind = -55.);

N_start = 3*10^5; @time pop_gen6 = Genie.run_dir_evol_nucleo(tem_DNA_seq, N_start, h, J;
                   rounds = 6,
                   seq_reads = N_start,
                   temp = 8.,  
                   mu = 0.03, mu_bind = -55.);


n_seq_fantini = 30000; 

@time fantini_exp = [Genie.analyze_target_msa(tem_dir_msa_6, tem, h, J, 6; n_seq = n_seq_fantini),
    Genie.analyze_target_msa(tem_dir_msa_12, tem, h, J, 12; n_seq = n_seq_fantini)];

fantini_mcmc = [Genie.analyze_msa(mcmc6.msa, tem_dir_msa_6, tem, h, J, 6; n_seq = n_seq_fantini),
    Genie.analyze_msa(mcmc12.msa, tem_dir_msa_12, tem, h, J, 12; n_seq = n_seq_fantini)];

fantini_popgen = [Genie.analyze_msa(pop_gen6.final_msa, tem_dir_msa_6, tem, h, J, 6; n_seq = n_seq_fantini),
    Genie.analyze_msa(pop_gen12.final_msa, tem_dir_msa_12, tem, h, J, 12; n_seq = n_seq_fantini)];




@save "../data_directed_evolution/data_for_figs_new.jld2" fantini_exp fantini_mcmc fantini_popgen stiffler_exp stiffler_mcmc stiffler_popgen   aac_exp aac_mcmc aac_popgen

matrix2fasta("../data_directed_evolution/silico/pop_gen12.fa", Int8.(pop_gen12.final_msa'))
matrix2fasta("../data_directed_evolution/silico/pop_gen6.fa", Int8.(pop_gen6.final_msa'))



matrix2fasta("../data_directed_evolution/silico/mcmc20.fa", Int8.(mcmc20.msa'))
matrix2fasta("../data_directed_evolution/silico/mcmc12.fa", Int8.(mcmc12.msa'))
matrix2fasta("../data_directed_evolution/silico/mcmc10.fa", Int8.(mcmc10.msa'))
matrix2fasta("../data_directed_evolution/silico/mcmc6.fa", Int8.(mcmc6.msa'))

matrix2fasta("../data_directed_evolution/silico/pop_gen20.fa", Int8.(pop_gen20.final_msa'))

matrix2fasta("../data_directed_evolution/silico/pop_gen10.fa", Int8.(pop_gen10.final_msa'))




matrix2fasta("../data_directed_evolution/silico/mcmc8.fa", Int8.(mcmc8.msa'))
matrix2fasta("../data_directed_evolution/silico/pop_gen8.fa", Int8.(pop_gen8.final_msa'))






## MCMC on the tree

In [ ]:
ENV["PYTHON"] = "" 
using Pkg
Pkg.build("PyCall")  # Rebuild PyCall to use the internal Python
using Revise, PhyloTools, DelimitedFiles, DCAUtils, JLD2, PyPlot, Statistics, LinearAlgebra, TreeTools
import KitMSA: fasta2matrix, matrix2fasta, cod2amino


h_tmp, J_tmp = read_par_BM_lettersave("../model_natural_PSE.dat");
h = set_max_field_to_0save(h_tmp);
J_tmp2 = symmetrize_Jsave(J_tmp); 
J = permutedims(J_tmp2, [1,3,2,4]);


h_tmp, J_tmp = read_par_BM_0gapsave("../data_directed_evolution/Parameters_conv_AAC6.dat");
h_aac = set_max_field_to_0save(h_tmp);
J_tmp2 = symmetrize_Jsave(J_tmp); 
J_aac = permutedims(J_tmp2, [1,3,2,4]);


wt_tem = join(readdlm("../Gen.jl/data/alignments/refs/TEM1_pfam_DNA.fasta", skipstart = 1));
L_big = round(Int,length(wt_tem)/3)
tem_DNA_seq = [wt_tem[((i-1)*3 +1):(i*3)] for i in 3:L_big-1];
tem = [cod2amino[x] for x in tem_DNA_seq];



wt_pse = join(readdlm("../Gen.jl/data/alignments/refs/PSE1_pfam_DNA.fasta", skipstart = 1));
L_big = round(Int,length(wt_pse)/3)
pse_DNA_seq = [wt_pse[((i-1)*3 +1):(i*3)] for i in 3:L_big-1];
pse = [cod2amino[x] for x in pse_DNA_seq];


wt_aac = join(readdlm("../Gen.jl/data/alignments/refs/AAC6_DNA_pfam.fasta", skipstart = 1));
L_big_aac = round(Int,length(wt_aac)/3)
aac_DNA_seq = [wt_aac[((i-1)*3 +1):(i*3)] for i in 1:L_big_aac];
aac = [cod2amino[x] for x in aac_DNA_seq];


@load "../data_directed_evolution/data_dir_evol_amino_tem.jld2"


@load "../data_directed_evolution/data_dir_evol_amino_pse.jld2"


@load "../data_directed_evolution/data_dir_evol_amino_aac.jld2"
    


In [ ]:
######### FANTINI ################


fantini_tree_file = "../data_directed_evolution/TreeSingleThread_TEM.nwk"
fantini_tree = read_tree(fantini_tree_file, node_data_type = Seq);
@time fantini_branch_d = mean([distance(fantini_tree.root, a) for a in leaves(fantini_tree)]);
fantini_mu = (95/199) / fantini_branch_d;



@time fantini_res12 = run_evolution_ontree(tem_DNA_seq, fantini_tree_file, 
    h, J, temp = 1., mu = fantini_mu, p = 0.); 
fantini_mcmc_tree12 = Int.(msa_from_leafs(fantini_res12)); 


@time fantini_res6 = run_evolution_ontree(tem_DNA_seq, fantini_tree_file, 
    h, J, temp = 1., mu = fantini_mu/2, p = 0.); 
fantini_mcmc_tree6 = Int.(msa_from_leafs(fantini_res6)); 




n_seq_fantini = 30000; 

fantini_mcmc_tree = [PhyloTools.analyze_msa(Int.(fantini_mcmc_tree6), Int.(tem_dir_msa_6), Int.(tem), h, J, 6; n_seq = n_seq_fantini),
    PhyloTools.analyze_msa(Int.(fantini_mcmc_tree12), Int.(tem_dir_msa_12), Int.(tem), h, J, 12; n_seq = n_seq_fantini)];



#### AAC6 ###############


aac_tree_file  = "../data_directed_evolution/TreeSingleThread_PSE_round20.nwk"
aac_tree = read_tree(aac_tree_file, node_data_type = Seq);
@time aac_branch_d = mean([distance(aac_tree.root, a) for a in leaves(aac_tree)]);
aac_mu = (40/117) / aac_branch_d;


@time aac_res8 = run_evolution_ontree(aac_DNA_seq, aac_tree_file, 
    h_aac, J_aac, temp = 1.6, mu = aac_mu, p = 0.); 
aac_mcmc_tree8 = Int.(msa_from_leafs(aac_res8)); 


n_seq_aac = 40000; 
aac_mcmc_tree = PhyloTools.analyze_msa(Int.(aac_mcmc_tree8), Int.(aac_dir_msa_8), Int.(aac), h_aac, J_aac, 
    8,  n_seq = n_seq_aac);






############# STIFFLER #####################

stiffler_tree_file  = "../data_directed_evolution/TreeSingleThread_PSE_round20.nwk"
stiffler_tree = read_tree(stiffler_tree_file, node_data_type = Seq);
@time stiffler_branch_d = mean([distance(stiffler_tree.root, a) for a in leaves(stiffler_tree)]);
stiffler_mu = (125/199) / stiffler_branch_d;


@time stiffler_res20 = run_evolution_ontree(pse_DNA_seq, stiffler_tree_file, 
    h, J, temp = 1.3, mu = stiffler_mu, p = 0.); 
stiffler_mcmc_tree20 = Int.(msa_from_leafs(stiffler_res20)); 



@time stiffler_res10 = run_evolution_ontree(pse_DNA_seq, stiffler_tree_file, 
    h, J, temp = 1.3, mu = stiffler_mu/2, p = 0.); 
stiffler_mcmc_tree10 = Int.(msa_from_leafs(stiffler_res10)); 

n_seq_stiffler = 40000;

stiffler_mcmc_tree = [PhyloTools.analyze_msa(Int.(stiffler_mcmc_tree10), Int.(pse_dir_msa_10), Int.(pse), 
        h, J, 10; n_seq = n_seq_stiffler),
    PhyloTools.analyze_msa(Int.(stiffler_mcmc_tree20), Int.(pse_dir_msa_20), Int.(pse), h, J, 20; n_seq = n_seq_stiffler)];

    
@save "../data_directed_evolution/data_for_figs_tree_new.jld2" fantini_mcmc_tree stiffler_mcmc_tree aac_mcmc_tree


matrix2fasta("../data_directed_evolution/silico/stiffler_mcmc_tree20.fa", Int8.(stiffler_mcmc_tree20'))
matrix2fasta("../data_directed_evolution/silico/stiffler_mcmc_tree10.fa", Int8.(stiffler_mcmc_tree10'))



matrix2fasta("../data_directed_evolution/silico/fantini_mcmc_tree12.fa", Int8.(fantini_mcmc_tree12'))
matrix2fasta("../data_directed_evolution/silico/fantini_mcmc_tree6.fa", Int8.(fantini_mcmc_tree12'))
matrix2fasta("../data_directed_evolution/silico/aac_mcmc_tree8.fa", Int8.(aac_mcmc_tree8'))




## POPGEN mcmc inspired

In [ ]:
@load "../data_directed_evolution/data_for_figs_new.jld2";
@load "../data_directed_evolution/data_for_figs_tree_new.jld2";


N_start = 10^5; @time pop_gen2_20 = Genie.run_dir_evol_nucleo_mcmc_inspired(pse_DNA_seq, N_start, h, J;
                   rounds = 20,
                   seq_reads = N_start,
                   temp = 45.,  
                   mu = 0.001);

N_start = 10^5; @time pop_gen2_10 = Genie.run_dir_evol_nucleo_mcmc_inspired(pse_DNA_seq, N_start, h, J;
                   rounds = 10,
                   seq_reads = N_start,
                   temp = 45.,  
                   mu = 0.001);

n_seq_stiffler = 1000;

@time stiffler_popgen2 = [Genie.analyze_msa(pop_gen2_10.final_msa, pse_dir_msa_10, pse, h, J, 10; n_seq = n_seq_stiffler),
    analyze_msa(pop_gen2_20.final_msa, pse_dir_msa_20, pse, h, J, 20; n_seq = n_seq_stiffler)];




# Parameters
n_bins = 20
n_points = 1000
colors = ["black", "red", "green", "pink","blue"]
labels = ["Exp", "MCMC", "Pop Gen", "Pop Gen 2", "MCMC on tree"]


# Create figure and 2x2 subplots
fig, axs = subplots(2, 2, figsize=(10, 8), sharex=false, sharey=true)



axs[1,1].hist(stiffler_exp[1].ham, histtype = "step", color = colors[1], linewidth = 2, bins = 20, density = true)
axs[1,1].hist(stiffler_mcmc[1].ham, histtype = "step", color = colors[2], linewidth = 2, bins = 20, density = true)
axs[1,1].hist(stiffler_popgen[1].ham, histtype = "step", color = colors[3], linewidth = 2, bins = 20, density = true)
axs[1,1].hist(stiffler_popgen2[1].ham, histtype = "step", color = colors[4], linewidth = 2, bins = 20, density = true)
axs[1,1].hist(stiffler_mcmc_tree[1].ham, histtype = "step", color = colors[5], linewidth = 2, bins = 20, density = true)
axs[1,1].set_yscale("log")
axs[1,1].set_title("Stiffler Round 10")
axs[1,1].set_xlabel("Hamming distance from wt")


axs[1,2].hist(stiffler_exp[2].ham, histtype = "step", color = colors[1], linewidth = 2, bins = 20, density = true)
axs[1,2].hist(stiffler_mcmc[2].ham, histtype = "step", color = colors[2], linewidth = 2, bins = 20, density = true)
axs[1,2].hist(stiffler_popgen[2].ham, histtype = "step", color = colors[3], linewidth = 2, bins = 20, density = true)
axs[1,2].hist(stiffler_popgen2[2].ham, histtype = "step", color = colors[4], linewidth = 2, bins = 20, density = true)
axs[1,2].hist(stiffler_mcmc_tree[2].ham, histtype = "step", color = colors[5], linewidth = 2, bins = 20, density = true)
axs[1,2].set_yscale("log")
axs[1,2].set_title("Stiffler Round 20")
axs[1,2].set_xlabel("Hamming distance from wt")





axs[2,1].hist(stiffler_exp[1].en, histtype = "step", color = colors[1], linewidth = 2, bins = 20, density = true)
axs[2,1].hist(stiffler_mcmc[1].en, histtype = "step", color = colors[2], linewidth = 2, bins = 20, density = true)
axs[2,1].hist(stiffler_popgen[1].en, histtype = "step", color = colors[3], linewidth = 2, bins = 20, density = true)
axs[2,1].hist(stiffler_popgen2[1].en, histtype = "step", color = colors[4], linewidth = 2, bins = 20, density = true)
axs[2,1].hist(stiffler_mcmc_tree[1].en, histtype = "step", color = colors[5], linewidth = 2, bins = 20, density = true)
axs[2,1].set_yscale("log")
axs[2,1].set_xlabel("Energy")


axs[2,2].hist(stiffler_exp[2].en, histtype = "step", color = colors[1], linewidth = 2, bins = 20, density = true)
axs[2,2].hist(stiffler_mcmc[2].en, histtype = "step", color = colors[2], linewidth = 2, bins = 20, density = true)
axs[2,2].hist(stiffler_popgen[2].en, histtype = "step", color = colors[3], linewidth = 2, bins = 20, density = true)
axs[2,2].hist(stiffler_popgen2[2].en, histtype = "step", color = colors[4], linewidth = 2, bins = 20, density = true)
axs[2,2].hist(stiffler_mcmc_tree[2].en, histtype = "step", color = colors[5], linewidth = 2, bins = 20, density = true)
axs[2,2].set_yscale("log")
axs[2,2].set_xlabel("Energy")



# Create a single shared legend outside the plots
fig.legend(labels, loc="upper center", ncol=5, bbox_to_anchor=(0.5, 0.95))

# Common axis labels
fig.text(0.02, 0.5, "Density", va="center", rotation="vertical", fontsize=12)

# Adjust layout to make room for labels and legend
fig.tight_layout(rect=[0.05, 0.05, 0.95, 0.92])
show()

savefig("../stiffler_popgen_mcmc_inspired.png", dpi = 300) 




## Gridsearch MCMC on the tree

In [ ]:
stiffler_tree_file  = "../data_directed_evolution/TreeSingleThread_PSE_round20.nwk"
stiffler_tree = read_tree(stiffler_tree_file, node_data_type = Seq);
@time stiffler_branch_d = mean([distance(stiffler_tree.root, a) for a in leaves(stiffler_tree)]);

n_steps = [100, 110, 120, 130, 140, 150];
temps = [1., 1.1, 1.2, 1.3, 1.4, 1.5];




mus = [(x/199) / stiffler_branch_d  for x in n_steps];



n_seq_stiffler = 10000
@time target = analyze_target_msa(pse_dir_msa_20, pse, h, J, 20; n_seq = n_seq_stiffler);
en_target = mean(target.en)[1]; ham_target = mean(target.ham)[1]; pair_ham_target = mean(target.pair_ham)[1];

# Initialize score matrices before the loop
score_ham = zeros(Float64, length(mus), length(temps))
score_en = zeros(Float64, length(mus), length(temps))
score_pair_ham = zeros(Float64, length(mus), length(temps))
#score_corr = zeros(Float64, length(mus), length(temps))
res_gridsearch = [];


for i in 1:length(mus)
    for j in 1:length(temps)
        @time stiffler_res20 = run_evolution_ontree(pse_DNA_seq, stiffler_tree_file, 
                h, J, temp = temps[j], mu = mus[i], p = 0.); 
        msa = Int.(msa_from_leafs(stiffler_res20))
        @time matrix2fasta("../data_directed_evolution/gridsearch_mcmc_tree/stiffler_round20_T$(temps[j])steps$(n_steps[i]).fa",
            Int8.(msa')); 
        @time perf = PhyloTools.analyze_target_msa(Int.(msa), Int.(pse), h, J, 20; n_seq = n_seq_stiffler);
        ham = mean(perf.ham)[1]; en = mean(perf.en)[1]; pair_ham = mean(perf.pair_ham)[1];
        
        score_ham[i,j] = abs((ham - ham_target)/ham_target)
        score_en[i,j] = abs((en - en_target)/en_target)
        score_pair_ham[i,j] = abs((pair_ham - pair_ham_target)/pair_ham_target)
        push!(res_gridsearch, perf)
        
        println("Loading $(i*j/36)%")
    end
end

@save "../data_directed_evolution/scores_gridsearch_mcmc_tree.jld2" score_en score_ham score_pair_ham temps n_steps
        

        
        

## Sample to sample fluctuations

In [ ]:
using Statistics

function pairwise_ham_dist_all(msa::Array{Int,2}; all = false)
    L,M = size(msa)
    new_msa = msa
    println("new")
    res = []
    for i in 1:M
        for j in i+1:M
            push!(res, ham_dist(new_msa[:, i], new_msa[:, j]))
        end
    end
    return all ? res : mean(res)
end

fantini_tree_file = "../data_directed_evolution/TreeSingleThread_TEM.nwk"
fantini_tree = read_tree(fantini_tree_file, node_data_type = Seq);
@time fantini_branch_d = mean([distance(fantini_tree.root, a) for a in leaves(fantini_tree)]);
fantini_mu = (95/199) / fantini_branch_d;




msas = []
for i in 1:5
@time fantini_res12 = run_evolution_ontree(tem_DNA_seq, fantini_tree_file, 
    h, J, temp = 1., mu = fantini_mu, p = 0.); 
push!(msas, Int.(msa_from_leafs(fantini_res12)))
end

pair_hams = [];
for i in 1:5
    push!(pair_hams, pairwise_ham_dist_all(msas[i], all = true) ./ 199)
end

pair_hams_dist = hcat(pair_hams...);

mean_pair_hams = mean(pair_hams_dist, dims = 2)[:];
var_pair_hams = var(pair_hams_dist, dims = 2)[:];


close("all"); plt.scatter(mean_pair_hams, var_pair_hams, alpha = 0.05); plt.xlabel("Mean Pairwise Hamming"
    ); plt.ylabel("Variance Pairwise Hamming"); plt.savefig("../sample2sample_tree.png")


close("all"); 
for i in 1:5
    plt.hist(pair_hams[i], histtype = "step", density = true)
end
plt.yscale("log")
plt.xlabel("Pairwise Hamming distance")
savefig("../sample2sample_pairwise_tree.png")
  


In [ ]:
using Statistics

function pairwise_ham_dist_all(msa::Array{Int,2}; all = false)
    L,M = size(msa)
    new_msa = msa
    println("new")
    res = []
    for i in 1:M
        for j in i+1:M
            push!(res, ham_dist(new_msa[:, i], new_msa[:, j]))
        end
    end
    return all ? res : mean(res)
end


stiffler_tree_file  = "../data_directed_evolution/TreeSingleThread_PSE_round20.nwk"
stiffler_tree = read_tree(stiffler_tree_file, node_data_type = Seq);
@time stiffler_branch_d = mean([distance(stiffler_tree.root, a) for a in leaves(stiffler_tree)]);
stiffler_mu = (125/199) / stiffler_branch_d;



msas = []
for i in 1:5
    @time stiffler_res20 = run_evolution_ontree(pse_DNA_seq, stiffler_tree_file, 
    h, J, temp = 1.3, mu = stiffler_mu, p = 0.); 
    push!(msas, Int.(msa_from_leafs(stiffler_res20)))
end

pair_hams = [];
for i in 1:5
    push!(pair_hams, PhyloTools.pairwise_ham_dist(msas[i], n_seq = 30000, all = true) ./ 199)
end

pair_hams_dist = hcat(pair_hams...);

mean_pair_hams = mean(pair_hams_dist, dims = 2)[:];
var_pair_hams = var(pair_hams_dist, dims = 2)[:];


close("all"); plt.scatter(mean_pair_hams, var_pair_hams, alpha = 0.05); plt.xlabel("Mean Pairwise Hamming"
    ); plt.ylabel("Variance Pairwise Hamming"); plt.savefig("../pse_sample2sample_tree.png")


close("all"); 
for i in 1:5
    plt.hist(pair_hams[i], histtype = "step", density = true)
end
plt.yscale("log")
plt.xlabel("Pairwise Hamming distance")
savefig("../pse_sample2sample_pairwise_tree.png")
    

## ASR reconstruction error

In [ ]:
using Statistics

function pairwise_ham_dist_all(msa::Array{Int,2}; all = false)
    L,M = size(msa)
    new_msa = msa
    println("new")
    res = []
    for i in 1:M
        for j in i+1:M
            push!(res, ham_dist(new_msa[:, i], new_msa[:, j]))
        end
    end
    return all ? res : mean(res)
end


@time fantini_res12 = run_evolution_ontree(tem_DNA_seq, fantini_tree_file, 
    h, J, temp = 1., mu = fantini_mu, p = 0.); 

matrix2fasta("../data_directed_evolution/ground_truth_fantini12.fa", Int8.(msa_from_leafs(fantini_res12)'))


gt_msa = read_fasta_alignment("../data_directed_evolution/ground_truth_fantini12.fa", 0.9)

fantini_tree_file = "../data_directed_evolution/TreeSingleThread_FantiniGT.nwk"
fantini_tree = read_tree(fantini_tree_file, node_data_type = Seq);
@time fantini_branch_d = mean([distance(fantini_tree.root, a) for a in leaves(fantini_tree)]);
fantini_mu = (95/199) / fantini_branch_d;

@time fantini_res12 = run_evolution_ontree(tem_DNA_seq, fantini_tree_file, 
    h, J, temp = 1., mu = fantini_mu, p = 0.); 

rec_msa = Int.(msa_from_leafs(fantini_res12))

@time rec_pair = pairwise_ham_dist_all(rec_msa, all = true) ./ 199
@time gt_pair = pairwise_ham_dist_all(Int.(gt_msa), all = true) ./ 199



close("all"); plt.hist(rec_pair,histtype = "step", linewidth = 2, 
    density = true, label = "Reconstructed");plt.hist(gt_pair,histtype = "step", linewidth = 2, 
    density = true, label = "Ground truth");plt.xlabel("Pairwise hamming distance"
    ); plt.yscale("log"); plt.legend(); savefig("../fantini_reconstruction_error_ASR.png")

            

In [ ]:
using Statistics

function pairwise_ham_dist_all(msa::Array{Int,2}; all = false)
    L,M = size(msa)
    new_msa = msa
    println("new")
    res = []
    for i in 1:M
        for j in i+1:M
            push!(res, ham_dist(new_msa[:, i], new_msa[:, j]))
        end
    end
    return all ? res : mean(res)
end

stiffler_tree_file  = "../data_directed_evolution/TreeSingleThread_PSE_round20.nwk"
stiffler_tree = read_tree(stiffler_tree_file, node_data_type = Seq);
@time stiffler_branch_d = mean([distance(stiffler_tree.root, a) for a in leaves(stiffler_tree)]);
stiffler_mu = (125/199) / stiffler_branch_d;



@time stiffler_res20 = run_evolution_ontree(pse_DNA_seq, stiffler_tree_file, 
    h, J, temp = 1.3, mu = stiffler_mu, p = 0.); 

matrix2fasta("../data_directed_evolution/ground_truth_stiffler20.fa", 
    Int.(msa_from_leafs(stiffler_res20)'))






gt_msa = read_fasta_alignment("../data_directed_evolution/ground_truth_stiffler20.fa", 0.9)


stiffler_tree_file  = "../data_directed_evolution/TreeSingleThread_StifflerGT.nwk"
stiffler_tree = read_tree(stiffler_tree_file, node_data_type = Seq);
@time stiffler_branch_d = mean([distance(stiffler_tree.root, a) for a in leaves(stiffler_tree)]);
stiffler_mu = (125/199) / stiffler_branch_d;


@time stiffler_res20 = run_evolution_ontree(pse_DNA_seq, stiffler_tree_file, 
    h, J, temp = 1., mu = fantini_mu, p = 0.); 

rec_msa = Int.(msa_from_leafs(stiffler_res20))

@time rec_pair = pairwise_ham_dist(rec_msa, n_seq = 30000, all = true) ./ 199
@time gt_pair = pairwise_ham_dist(Int.(gt_msa), n_seq = 30000, all = true) ./ 199



close("all"); plt.hist(rec_pair,histtype = "step", linewidth = 2, 
    density = true, label = "Reconstructed");plt.hist(gt_pair,histtype = "step", linewidth = 2, 
    density = true, label = "Ground truth");plt.xlabel("Pairwise hamming distance"
    ); plt.yscale("log"); plt.legend(); savefig("../stiffler_reconstruction_error_ASR.png")

            

## Long term evolution

In [ ]:



start_msa = hcat([pse_DNA_seq for i in 1:10^3]...); 
    @time mcmc = run_evolution(start_msa, 
    h, 
    J,
    p = 0., 
    temp = 1., 
    N_points = 50, 
    N_steps = 10^6, 
    verbose = true);


N_start = 10^5; @time popgen = Genie.run_dir_evol_nucleo(pse_DNA_seq, N_start, h, J;
                   rounds = 12500,
                   seq_reads = N_start,
                   temp = 8.,
                   each_step = 125,
                   mu = 0.03, mu_bind = -55.);


nat_msa = read_fasta_alignment("../Gen.jl/data/alignments/natural/PF13354_noinsert_max19gaps_nodupl_noclose.faa", 0.9)[3:L_big-1,:];



pc_nat, pc_sil = check_pca(Int8.(nat_msa), Int8.(popgen.step_msa[end]));
pc_nat, pc_pse = check_pca(Int8.(nat_msa), hcat([pse for _ in 1:2]...));
pc_nat, pc_mcmc = check_pca(Int8.(nat_msa), Int8.(mcmc.step_msa[end]));

close("all")
plt.scatter(pc_nat[:,1], pc_nat[:,2], label = "Natural sequences")
plt.scatter(pc_mcmc[:,1], pc_mcmc[:,2], label = "MCMC (nucleo) long")
plt.scatter(pc_sil[:,1], pc_sil[:,2], label = "PopGen (nucleo) long")
plt.scatter(pc_pse[:,1], pc_pse[:,2], label = "PSE")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
savefig("../pca_beta_popgen_mcmc.png")


L_big = 202; 
nat_msa = read_fasta_alignment("../Gen.jl/data/alignments/natural/PF13354_noinsert_max19gaps_nodupl_noclose.faa", 
        0.9)[3:L_big-1,:];


Genie.check_timescales(mcmc.step_msa, nat_msa, h, J, mcmc.steps, "../timescales_mcmc.png")
    
    

Genie.check_timescales(popgen.step_msa, nat_msa, h, J, popgen.steps, "../timescales_popgen.png")
    
     
    
for i in 1:length(popgen.steps)
    matrix2fasta("../data_directed_evolution/long_term_popgen/round$(popgen.steps[i])T8mu_bindminus55mu0.03.fa",
        Int8.(popgen.step_msa[i]'))
end
    


In [ ]:
N_start = 10^5; @time popgen2 = Genie.run_dir_evol_nucleo_mcmc_inspired(pse_DNA_seq, N_start, h, J;
                   rounds = 20000,
                   seq_reads = N_start,
                   temp = 4.,
                   mu = 0.03);

matrix2fasta("../data_directed_evolution/long_term_popgen2/round20000T4mu0.03.fa",
        Int8.(popgen2.final_msa'))
    


L_big = 202; 
nat_msa = read_fasta_alignment("../Gen.jl/data/alignments/natural/PF13354_noinsert_max19gaps_nodupl_noclose.faa", 
        0.9)[3:L_big-1,:];

pc_nat, pc_sil = check_pca(Int8.(nat_msa), Int8.(popgen2.final_msa));
pc_nat, pc_pse = check_pca(Int8.(nat_msa), hcat([pse for _ in 1:2]...));
#pc_nat, pc_mcmc = check_pca(Int8.(nat_msa), Int8.(mcmc.step_msa[end]));

close("all")
plt.scatter(pc_nat[:,1], pc_nat[:,2], label = "Natural sequences")
#plt.scatter(pc_mcmc[:,1], pc_mcmc[:,2], label = "MCMC (nucleo) long")
plt.scatter(pc_sil[:,1], pc_sil[:,2], label = "PopGen (nucleo) long")
plt.scatter(pc_pse[:,1], pc_pse[:,2], label = "PSE")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
savefig("../pca_beta_popgen2.png")



Genie.check_timescales([hcat([pse for i in 1:1000]...), popgen2.final_msa], nat_msa, 
    h, J, [1, 10000], "../timescales_popgen2.png")
    
     
    


In [ ]:
############### MCMC LONG on ROUND20 TREE ###################



stiffler_tree_file  = "../data_directed_evolution/TreeSingleThread_PSE_round20.nwk"
stiffler_tree = read_tree(stiffler_tree_file, node_data_type = Seq);
@time stiffler_branch_d = mean([distance(stiffler_tree.root, a) for a in leaves(stiffler_tree)]);
n_steps = 10^5
stiffler_mu = (n_steps/199) / stiffler_branch_d;


@time stiffler_res = run_evolution_ontree(pse_DNA_seq, stiffler_tree_file, 
    h, J, temp = 1., mu = stiffler_mu, p = 0.); 
stiffler_mcmc_tree = Int.(msa_from_leafs(stiffler_res)); 

matrix2fasta("../data_directed_evolution/long_term_mcmc_tree/temp1.0steps$(n_steps).fa",
        Int8.(stiffler_mcmc_tree'))

nat_msa = read_fasta_alignment("../Gen.jl/data/alignments/natural/PF13354_noinsert_max19gaps_nodupl_noclose.faa", 
               0.9)[3:L_big-1,:]

@time pair_ham_mcmc_tree = PhyloTools.pairwise_ham_dist(stiffler_mcmc_tree, n_seq = 10000, all = true) ./ 199;
@time pair_ham_nat = PhyloTools.pairwise_ham_dist(nat_msa, n_seq = 10000, all = true)  ./ 199 ;

close("all"); plt.hist(pair_ham_mcmc_tree, histtype = "step", 
    density = true, label = "MCMC tree round20"); plt.hist(pair_ham_nat, histtype = "step", 
    density = true, label = "Nat"); plt.xlabel("Pairwise hamming distance"
    ); plt.yscale("log"); plt.legend();savefig("../pairwise_long_mcmc_tree_round20.png")


############ do this part in Genie #######################


n_steps = 10^5
nat_msa = read_fasta_alignment("../Gen.jl/data/alignments/natural/PF13354_noinsert_max19gaps_nodupl_noclose.faa", 
        0.9)[3:L_big-1,:];

init_msa = hcat([pse for i in 1:size(pse_dir_msa_20,2)]...)
final_msa = read_fasta_alignment("../data_directed_evolution/long_term_mcmc_tree/temp1.0steps$(n_steps).fa", 0.9); 

Genie.check_timescales([init_msa, final_msa], nat_msa, h, J, 
    [1, n_steps], "../timescales_mcmc_tree.png")

pc_nat, pc_sil = check_pca(Int8.(nat_msa), Int8.(final_msa));
pc_nat, pc_pse = check_pca(Int8.(nat_msa), hcat([pse for _ in 1:2]...));
pc_nat, pc_mcmc = check_pca(Int8.(nat_msa), Int8.(final_msa));

close("all")
plt.scatter(pc_nat[:,1], pc_nat[:,2], label = "Natural sequences")
plt.scatter(pc_mcmc[:,1], pc_mcmc[:,2], label = "MCMC tree of round 20 (nucleo) long")
plt.scatter(pc_sil[:,1], pc_sil[:,2], label = "PopGen (nucleo) long")
plt.scatter(pc_pse[:,1], pc_pse[:,2], label = "PSE")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
savefig("../pca_beta_popgen_mcmc_tree.png")




In [ ]:

############### MCMC LONG on NATURAL TREE ###################



stiffler_tree_file  = "../data_directed_evolution/TreeSingleThread_BetaLactNatural.nwk"
stiffler_tree = read_tree(stiffler_tree_file, node_data_type = Seq);
@time stiffler_branch_d = mean([distance(stiffler_tree.root, a) for a in leaves(stiffler_tree)]);
n_steps = 10^7
stiffler_mu = (n_steps/199) / stiffler_branch_d;


@time stiffler_res = run_evolution_ontree(pse_DNA_seq, stiffler_tree_file, 
    h, J, temp = 1., mu = stiffler_mu, p = 0.); 
stiffler_mcmc_tree = Int.(msa_from_leafs(stiffler_res)); 

matrix2fasta("../data_directed_evolution/long_term_mcmc_tree_nat/temp1.0steps$(n_steps).fa",
        Int8.(stiffler_mcmc_tree'))

nat_msa = read_fasta_alignment("../Gen.jl/data/alignments/natural/PF13354_noinsert_max19gaps_nodupl_noclose.faa", 
               0.9)[3:L_big-1,:];

@time pair_ham_mcmc_tree = PhyloTools.pairwise_ham_dist(stiffler_mcmc_tree, n_seq = 10000, all = true) ./ 199;
@time pair_ham_nat = PhyloTools.pairwise_ham_dist(nat_msa, n_seq = 10000, all = true)  ./ 199 ;

close("all"); plt.hist(pair_ham_mcmc_tree, histtype = "step", 
    density = true, label = "MCMC tree nat"); plt.hist(pair_ham_nat, histtype = "step", 
    density = true, label = "Nat"); plt.xlabel("Pairwise hamming distance"
    ); plt.yscale("log"); plt.legend(); savefig("../pairwise_long_mcmc_tree_nat.png")



############ do this part in Genie #######################

n_steps = 10^7

nat_msa = read_fasta_alignment("../Gen.jl/data/alignments/natural/PF13354_noinsert_max19gaps_nodupl_noclose.faa", 
        0.9)[3:L_big-1,:];

init_msa = hcat([pse for i in 1:size(pse_dir_msa_20,2)]...)
final_msa = read_fasta_alignment("../data_directed_evolution/long_term_mcmc_tree_nat/temp1.0steps$(n_steps).fa", 0.9); 

Genie.check_timescales([init_msa, final_msa], nat_msa, h, J, 
    [1, n_steps], "../timescales_mcmc_tree_nat.png")

pc_nat, pc_sil = check_pca(Int8.(nat_msa), Int8.(final_msa));
pc_nat, pc_pse = check_pca(Int8.(nat_msa), hcat([pse for _ in 1:2]...));
pc_nat, pc_mcmc = check_pca(Int8.(nat_msa), Int8.(final_msa));

close("all")
plt.scatter(pc_nat[:,1], pc_nat[:,2], label = "Natural sequences")
plt.scatter(pc_mcmc[:,1], pc_mcmc[:,2], label = "MCMC tree of nat (nucleo) long")
plt.scatter(pc_sil[:,1], pc_sil[:,2], label = "PopGen (nucleo) long")
plt.scatter(pc_pse[:,1], pc_pse[:,2], label = "PSE")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.legend()
savefig("../pca_beta_popgen_mcmc_tree_nat.png")




In [ ]:
h_tmp, J_tmp = read_par_BM_0gapsave("../bmDCA_params.dat");
h_tmp, J_tmp = read_par_BM_lettersave("../model_natural_PSE.dat");
h72 = set_max_field_to_0save(h_tmp);
J_tmp2 = symmetrize_Jsave(J_tmp); 
J72 = permutedims(J_tmp2, [1,3,2,4]);


